In [1]:
import numpy as np
import pandas as pd
import csv
from time import time

# plots
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from datetime import datetime
import matplotlib.pyplot as plt

In [2]:
data_17_08 = pd.read_csv('../data/properati/properati-AR-2017-08-01-properties-sell.csv')
data_17_02 = pd.read_csv('../data/properati/properati-AR-2017-02-01-properties-sell.csv')
data_16_08 = pd.read_csv('../data/properati/properati-AR-2016-08-01-properties-sell.csv')
data_16_02 = pd.read_csv('../data/properati/properati-AR-2016-02-01-properties-sell.csv')
data_15_08 = pd.read_csv('../data/properati/properati-AR-2015-08-01-properties-sell.csv')
data_15_02 = pd.read_csv('../data/properati/properati-AR-2015-02-01-properties-sell.csv')
data_14_08 = pd.read_csv('../data/properati/properati-AR-2014-08-01-properties-sell.csv')
data_14_02 = pd.read_csv('../data/properati/properati-AR-2014-02-01-properties-sell.csv')
data_13_08 = pd.read_csv('../data/properati/properati-AR-2013-08-01-properties-sell.csv')

data_frames16_17 = [data_17_08, data_17_02, data_16_08, data_16_02]
train16_17 = pd.concat(data_frames16_17)

#Nos quedamos con las zonas de Capital Federal y GBA
train16_17 = train16_17.loc[(train16_17.state_name=='Capital Federal') | (train16_17.state_name.str.contains('G.B.A'))]

#Los dataframe del 2013, 2014, 2015 no tienen la columna state_name, todas las propiedades son de CABA y GBA
data_frames13_15 = [data_15_08,data_15_02,data_14_08,data_14_02,data_13_08]
train13_15 = pd.concat(data_frames13_15)

train13_15.rename(columns={'surface_in_m2':'surface_total_in_m2'}, inplace=True)

data_frames = [train16_17,train13_15]
train = pd.concat(data_frames)

In [3]:
train = train.loc[(train.property_type != 'store')]

train['created_on'] = pd.to_datetime(train['created_on'])
train = train.drop('image_thumbnail', axis=1)
train = train.drop('operation', axis=1)
train = train.drop('properati_url', axis=1)
train = train.drop('description', axis=1)
train = train.drop('place_with_parent_names', axis=1)
train = train.drop('title', axis=1)
train = train.drop('state_name', axis=1)
train = train.drop('property_type', axis=1)
train = train.drop('lat-lon', axis=1)
train = train.drop('id', axis=1)
train = train.drop('country_name', axis=1)
train = train.drop('extra', axis=1)
train = train.drop('price_aprox_local_currency', axis=1)
train = train.drop('price_per_m2', axis=1)
train = train.drop('currency', axis=1)
train = train.drop('created_on', axis=1)
train = train.drop('geonames_id', axis=1)
train = train.drop('lat', axis=1)
train = train.drop('lon', axis=1)
train = train.drop('price', axis=1)
train = train.drop('surface_covered_in_m2', axis=1)

In [4]:
train['expenses'] = pd.to_numeric(train['expenses'], errors='coerce')
train['price_usd_per_m2'] = pd.to_numeric(train['price_usd_per_m2'], errors='coerce')
train['surface_total_in_m2'] = pd.to_numeric(train['surface_total_in_m2'], errors='coerce')
train['floor'] = pd.to_numeric(train['floor'], errors='coerce')
train['rooms'] = pd.to_numeric(train['rooms'], errors='coerce')

train[['price_usd_per_m2', 'rooms','surface_total_in_m2']] \
= train[['price_usd_per_m2', 'rooms','surface_total_in_m2']].fillna(value=0)
train[['expenses']] = train[['expenses']].fillna(value=0)
train[['floor']] = train[['floor']].fillna(value=0)
train[['price_aprox_usd']] = train[['price_aprox_usd']].fillna(value=0)

#Elimino las filas que no indican el precio (o indican el precio pero ningun otro feature)
train=train.loc[train['price_aprox_usd'] > 0]
train=train.loc[(train['expenses'] > 0) | (train['floor'] > 0) | (train['price_usd_per_m2'] > 0) | (train['rooms'] > 0) \
            | (train['surface_total_in_m2'] > 0)]

In [5]:
# cargamos el archivo de test
test  = pd.read_csv("../data/properati/properati_dataset_testing_noprice.csv")

test = test.drop('description', axis=1)
test = test.drop('place_with_parent_names', axis=1)
test = test.drop('property_type', axis=1)
test = test.drop('lat-lon', axis=1)
test = test.drop('country_name', axis=1)
test = test.drop('operation', axis=1)
test = test.drop('created_on', axis=1)
test = test.drop('state_name', axis=1)
test = test.drop('lat', axis=1)
test = test.drop('lon', axis=1)
test = test.drop('surface_covered_in_m2', axis=1)
#test = test.drop('floor', axis=1)

test['expenses'] = pd.to_numeric(test['expenses'], errors='coerce')

test[['surface_total_in_m2','rooms','floor','expenses']] = \
test[['surface_total_in_m2','rooms','floor','expenses']].fillna(value=0)
test[['expenses']] = test[['expenses']].fillna(value=0)

In [6]:
df_places = pd.get_dummies(train['place_name'], drop_first=True)
df_Test_places = pd.get_dummies(test['place_name'], drop_first=True)
df_places=df_places.filter(df_Test_places.columns.values)

#Lugares que no estaban en el el set de entrenamiento
BarrioCerradoMalibu=pd.Series(0, df_places.index)
VillaBrown=pd.Series(0, df_places.index)
VillaNumancia=pd.Series(0, df_places.index)

df_places = pd.concat([df_places, BarrioCerradoMalibu, VillaBrown,VillaNumancia], axis=1)

train = pd.concat([train, df_places], axis=1)
train = train.drop('place_name', axis=1)

In [7]:
df_places = pd.get_dummies(test['place_name'], drop_first=True)
test = pd.concat([test, df_places], axis=1)
test = test.drop('place_name', axis=1)

In [8]:
train.rename(columns={0:'Barrio Cerrado Malibú',1:'VillaBrown',2:'VillaNumancia'}, inplace=True)

In [9]:
train.head(1)

,expenses,floor,price_aprox_usd,price_usd_per_m2,rooms,surface_total_in_m2,Abasto,Abril Club de Campo,Adrogué,Agronomía,...,Villa Tesei,Villa Udaondo,Villa Urquiza,Villa del Parque,Virrey del Pino,Wilde,William Morris,Barrio Cerrado Malibú,VillaBrown,VillaNumancia
0,0.0,0.0,120000.0,360.36036,0.0,333.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
train_label = []
train_data = []

train_label = train.iloc[:, 2]
train_data = train.iloc[:, train.columns != 'price_aprox_usd']
print("train_data set has {0[0]} rows and {0[1]} columns".format(train_data.shape))

train_data set has 419922 rows and 227 columns


In [11]:
#-----------------------------------------------------------------
# Modelo Random Forest Regressor
#-----------------------------------------------------------------
from sklearn.ensemble import RandomForestRegressor

In [12]:
modeloRFR=RandomForestRegressor(random_state=0, n_jobs=-1, n_estimators=100)

#modeloRFR.fit(train_data,train_label)

In [13]:
precision = cross_val_score(modeloRFR, X=train_data, y=train_label, scoring='neg_mean_squared_error', cv=4, n_jobs=-1)
print ("Mean squared error: {0:.5f}").format(-precision.mean())

Mean squared error: 26035325675.41708


In [14]:
modeloRFR.fit(train_data, train["price_aprox_usd"])

result = modeloRFR.predict(test)

In [16]:
def write_submission(test_data, prediction, file_output):
    
    archivo_entrada = open(test_data)
    entrada_csv = csv.reader(archivo_entrada)
    next(entrada_csv, None)  # skip the headers

    archivo_salida = open(file_output, 'w')
    submit_csv = csv.writer(archivo_salida)
    submit_csv.writerow(['id', 'price_usd'])

    for reg1, reg2  in zip(entrada_csv, prediction):
        linea = [reg1[0], round(reg2, 2)]
        submit_csv.writerow(linea)
    archivo_salida.close()

In [17]:
write_submission("../data/properati/properati_dataset_testing_noprice.csv", result, "../predicciones/RFR_dummy.csv")